# GDACS Data with pystac-monty

This notebook demonstrates how to use pystac-monty to process GDACS (Global Disaster Alert and Coordination System) data and visualize it using interactive maps. We'll:

1. Download GDACS data from the GDACS API
2. Convert the data to STAC items using pystac-monty
3. Display events on an interactive map
4. Explore the Monty STAC model and its metadata

Let's begin by importing the necessary libraries.

In [10]:
%load_ext autoreload
%autoreload 2

# Basic libraries
import json

# Visualization libraries
import folium
import requests

# STAC and pystac-monty
from pystac_monty.geocoding import WorldAdministrativeBoundariesGeocoder
from pystac_monty.sources.gdacs import GDACSDataSource, GDACSDataSourceType, GDACSTransformer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Download and Process GDACS Data

First, let's get a list of GDACS events and then download detailed data for a specific event.

In [11]:
# Define GDACS API parameters for event list
event_type = ""  # Flood events
from_date = "01-01-2023"  # Start date
to_date = "01-01-2024"    # End date
limit = 1000                # Limit the number of events

# Construct the GDACS API URL for event list
gdacs_event_list_url = f"https://www.gdacs.org/gdacsapi/api/events/geteventlist/SEARCH?eventtype={event_type}&fromdate={from_date}&todate={to_date}"

print(f"Downloading GDACS event list from {gdacs_event_list_url}...")
event_list_response = requests.get(gdacs_event_list_url)
event_list_data = event_list_response.text
print(f"Downloaded {len(event_list_data)} bytes of event list data")

# Parse the event list data
event_list_json = json.loads(event_list_data)
print(f"Number of events in the list: {len(event_list_json.get('features', []))}")
if limit:
    event_list_json["features"] = event_list_json["features"][:limit]
print(f"Number of events in the list after limiting: {len(event_list_json.get('features', []))}")

# Initialize the geocoder
geocoder = WorldAdministrativeBoundariesGeocoder("../tests/data-files/world-administrative-boundaries.fgb")

Downloaded 129949 bytes of event list data
Number of events in the list: 92
Number of events in the list after limiting: 92


## 2. Create STAC Items from GDACS Data

Now, let's transform the GDACS data into STAC items.

In [12]:
all_stac_items = []

# Loop on events
for selected_event in event_list_json.get("features", []):
    event_id = selected_event.get("properties", {}).get("eventid")
    event_type = selected_event.get("properties", {}).get("eventtype")

    # Download the main event data
    print(f"\nSelected event: ID={event_id}, Type={event_type}")
    gdacs_event_url = f"https://www.gdacs.org/gdacsapi/api/events/geteventdata?eventtype={event_type}&eventid={event_id}"
    print(f"\nDownloading GDACS event data from {gdacs_event_url}...")
    event_response = requests.get(gdacs_event_url)
    event_data = event_response.text
    print(f"Downloaded {len(event_data)} bytes of event data")
    event = json.loads(event_data)

    # Prepare to collect episode data
    episodes_data = []

    # Get all episodes of the event
    for episode in event.get("properties", {}).get("episodes", []):
        print(f"\nSelected episode: ID={episode.get('details', None)}")
        episode_url = episode.get("details", None)

        # Extract episode number from the URL
        # e.g. https://www.gdacs.org/gdacsapi/api/events/geteventdata?eventtype=FL&eventid=1102983&episodeid=2
        episode_id = episode_url.split("episodeid=")[1]

        # Construct the GDACS API URLs for detailed event data of the episode
        # e.g. https://www.gdacs.org/gdacsapi/api/events/getepisodedata?eventtype=FL&eventid=1102983&episodeid=1
        #      https://www.gdacs.org/gdacsapi/api/polygons/getgeometry?eventtype=FL&eventid=1102983&episodeid=2
        gdacs_episode_url = f"https://www.gdacs.org/gdacsapi/api/events/getepisodedata?eventtype={event_type}&eventid={event_id}&episodeid={episode_id}"
        gdacs_geometry_url = f"https://www.gdacs.org/gdacsapi/api/polygons/getgeometry?eventtype={event_type}&eventid={event_id}&episodeid={episode_id}"

        # Download episode event data
        print(f"\nDownloading GDACS episode data from {gdacs_episode_url}...")
        episode_response = requests.get(gdacs_episode_url)
        print(f"Downloaded {len(episode_response.text)} bytes of episode data")
        episode_data = episode_response.json()

        # Download episode geometry data
        print(f"Downloading GDACS geometry data from {gdacs_geometry_url}...")
        geometry_response = requests.get(gdacs_geometry_url)
        print(f"Downloaded {len(geometry_response.text)} bytes of geometry data")
        geometry_data = geometry_response.json() if geometry_response.text else None

        # Create episode data dictionary
        episode_dict = {GDACSDataSourceType.EVENT: (gdacs_episode_url, episode_data)}

        # Add geometry data if available
        if geometry_data:
            episode_dict[GDACSDataSourceType.GEOMETRY] = (gdacs_geometry_url, geometry_data)

        # Add to episodes list
        episodes_data.append(episode_dict)

    # Create a single GDACSDataSource with all episodes
    gdacs_data_source = GDACSDataSource(source_url=gdacs_event_url, data=event, episodes=episodes_data)

    # Initialize the transformer with the data source
    transformer = GDACSTransformer(gdacs_data_source, geocoder)

    # Create STAC items
    print("Creating STAC items from GDACS data...")
    try:
        stac_items = transformer.make_items()
        all_stac_items.extend(stac_items)
        print(f"Created {len(stac_items)} STAC items")
    except Exception as e:
        print(f"Error creating STAC items: {e}")
        continue


Selected event: ID=1016449, Type=DR

Downloaded 9352 bytes of event data

Selected episode: ID=https://www.gdacs.org/gdacsapi/api/events/getepisodedata?eventtype=DR&eventid=1016449&episodeid=1

Downloaded 9343 bytes of episode data
Downloaded 7594 bytes of geometry data

Selected episode: ID=https://www.gdacs.org/gdacsapi/api/events/getepisodedata?eventtype=DR&eventid=1016449&episodeid=2

Downloaded 9343 bytes of episode data
Downloaded 7629 bytes of geometry data

Selected episode: ID=https://www.gdacs.org/gdacsapi/api/events/getepisodedata?eventtype=DR&eventid=1016449&episodeid=3

Downloaded 9343 bytes of episode data
Downloaded 39977 bytes of geometry data

Selected episode: ID=https://www.gdacs.org/gdacsapi/api/events/getepisodedata?eventtype=DR&eventid=1016449&episodeid=4

Downloaded 9343 bytes of episode data
Downloaded 45119 bytes of geometry data

Selected episode: ID=https://www.gdacs.org/gdacsapi/api/events/getepisodedata?eventtype=DR&eventid=1016449&episodeid=5

Downloaded 

In [13]:
# Separate the STAC items by role
event_items = []
hazard_items = []
impact_items = []

for item in all_stac_items:
    roles = item.properties.get("roles", [])
    if "event" in roles:
        event_items.append(item)
    elif "hazard" in roles:
        hazard_items.append(item)
    elif "impact" in roles:
        impact_items.append(item)

print(f"Events: {len(event_items)}, Hazards: {len(hazard_items)}, Impacts: {len(impact_items)}")

Events: 86, Hazards: 973, Impacts: 128


## 3. Add Items to STAC Collections

Now, let's add the STAC items to their respective collections in a STAC API.

In [14]:
# Import STAC helper functions
import sys
sys.path.append('.')
from stac_helpers import (
    add_items_to_collection,
    check_collection_exists,
    check_stac_api_availability,
    create_collection_fallback,
    create_collection_from_file,
    delete_collection,
)

In [15]:
# Define the STAC API endpoint
# Replace with your actual STAC API endpoint
stac_api_url = "https://montandon-eoapi-stage.ifrc.org/stac"

# Define the collection IDs for each type of item
# These match the predefined collections in monty-stac-extension/examples
event_collection_id = "gdacs-events"
hazard_collection_id = "gdacs-hazards"
impact_collection_id = "gdacs-impacts"

# Define paths to the predefined collection definitions
event_collection_path = "../monty-stac-extension/examples/gdacs-events/gdacs-events.json"
hazard_collection_path = "../monty-stac-extension/examples/gdacs-hazards/gdacs-hazards.json"
impact_collection_path = "../monty-stac-extension/examples/gdacs-impacts/gdacs-impacts.json"

# Check if the STAC API is available
api_available = check_stac_api_availability(stac_api_url)

STAC API is available at https://montandon-eoapi-stage.ifrc.org/stac


In [16]:
# Check if the collections exist and create them if they don't
if api_available:
    # Check and create event collection if needed
    # delete_collection(stac_api_url, event_collection_id)
    event_collection_exists = check_collection_exists(stac_api_url, event_collection_id)
    if not event_collection_exists:
        print(f"\nAttempting to create collection '{event_collection_id}'...")
        event_collection_created = create_collection_from_file(stac_api_url, event_collection_path)
        if not event_collection_created:
            print("Trying fallback method to create event collection...")
            event_collection_created = create_collection_fallback(
                stac_api_url, event_collection_id, "GDACS events processed with pystac-monty", ["event", "source"]
            )
        event_collection_exists = event_collection_created

    # Check and create hazard collection if needed
    # delete_collection(stac_api_url, hazard_collection_id)
    hazard_collection_exists = check_collection_exists(stac_api_url, hazard_collection_id)
    if not hazard_collection_exists:
        print(f"\nAttempting to create collection '{hazard_collection_id}'...")
        hazard_collection_created = create_collection_from_file(stac_api_url, hazard_collection_path)
        if not hazard_collection_created:
            print("Trying fallback method to create hazard collection...")
            hazard_collection_created = create_collection_fallback(
                stac_api_url, hazard_collection_id, "GDACS hazards processed with pystac-monty", ["hazard", "source"]
            )
        hazard_collection_exists = hazard_collection_created

    # Check and create impact collection if needed
    if impact_items:
        # delete_collection(stac_api_url, impact_collection_id)
        impact_collection_exists = check_collection_exists(stac_api_url, impact_collection_id)
        if not impact_collection_exists:
            print(f"\nAttempting to create collection '{impact_collection_id}'...")
            impact_collection_created = create_collection_from_file(stac_api_url, impact_collection_path)
            if not impact_collection_created:
                print("Trying fallback method to create impact collection...")
                impact_collection_created = create_collection_fallback(
                    stac_api_url, impact_collection_id, "GDACS impacts processed with pystac-monty", ["impact", "source"]
                )
            impact_collection_exists = impact_collection_created
    else:
        impact_collection_exists = False

    if not (event_collection_exists and hazard_collection_exists and (not impact_items or impact_collection_exists)):
        print("\nWarning: One or more collections could not be created in the STAC API.")
        print("Some items may not be added to the STAC API.")
else:
    print("STAC API is not available. Skipping collection checks and creation.")

Collection 'gdacs-events' exists in the STAC API
Collection 'gdacs-hazards' exists in the STAC API
Collection 'gdacs-impacts' exists in the STAC API


In [17]:
# Add the items to their respective collections if the API is available
if api_available:
    if event_collection_exists:
        print("Adding event items to the collection...")
        event_success, event_failed = add_items_to_collection(stac_api_url, event_collection_id, event_items, overwrite=True)
    else:
        print("Skipping adding event items because the collection doesn't exist")
        event_success, event_failed = 0, len(event_items)

    if hazard_collection_exists:
        print("\nAdding hazard items to the collection...")
        hazard_success, hazard_failed = add_items_to_collection(stac_api_url, hazard_collection_id, hazard_items, overwrite=True)
    else:
        print("Skipping adding hazard items because the collection doesn't exist")
        hazard_success, hazard_failed = 0, len(hazard_items)

    if impact_items and impact_collection_exists:
        print("\nAdding impact items to the collection...")
        impact_success, impact_failed = add_items_to_collection(stac_api_url, impact_collection_id, impact_items)
    else:
        print("Skipping adding impact items because the collection doesn't exist or there are no impact items")
        impact_success, impact_failed = 0, len(impact_items)

    total_success = event_success + hazard_success + impact_success
    total_failed = event_failed + hazard_failed + impact_failed

    print(f"\nSummary: Added {total_success} items successfully, {total_failed} items failed")
else:
    print("Skipping adding items to collections because the API is not available")

Adding event items to the collection...
Processing batch 1 of 1 (86 items)
Item gdacs-event-1016449 already exists in the collection
Item gdacs-event-1016686 already exists in the collection
Item gdacs-event-1016682 already exists in the collection
Item gdacs-event-1016833 already exists in the collection
Item gdacs-event-1102407 already exists in the collection
Item gdacs-event-1408627 already exists in the collection
Added 86 items successfully, 0 items failed

Adding hazard items to the collection...
Processing batch 1 of 1 (973 items)
Item gdacs-hazard-1016449-1 already exists in the collection
Item gdacs-hazard-1016449-2 already exists in the collection
Item gdacs-hazard-1016449-3 already exists in the collection
Item gdacs-hazard-1016449-4 already exists in the collection
Item gdacs-hazard-1016449-5 already exists in the collection
Item gdacs-hazard-1016449-6 already exists in the collection
Item gdacs-hazard-1016449-7 already exists in the collection
Item gdacs-hazard-1016449-8 

## 4. Visualize GDACS Events on a Map

Let's create an interactive map to visualize the GDACS events.

In [18]:
# Create a map centered on the first event's location or a default location
if event_items:
    # Get the coordinates of the first event
    first_event = event_items[0]
    coords = first_event.geometry["coordinates"]
    center = [coords[1], coords[0]]  # [lat, lon]
else:
    # Default center (Madrid, Spain)
    center = [40.4168, -3.7038]

# Create the map
m = folium.Map(location=center, zoom_start=6)

# Add a marker cluster for the events
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each event
for item in event_items:
    # Get the coordinates
    coords = item.geometry["coordinates"]
    lat, lon = coords[1], coords[0]
    
    # Get event properties
    title = item.properties.get("title", "No title")
    description = item.properties.get("description", "No description")
    event_type = item.properties.get("monty:hazard_codes", ["Unknown"])[0]
    date = item.datetime.strftime("%Y-%m-%d")
    
    # Create popup content
    popup_content = f"<b>ID:</b> {item.id}<br>"
    popup_content += f"<b>Type:</b> {event_type}<br>"
    popup_content += f"<b>Date:</b> {date}<br>"
    popup_content += f"<b>Title:</b> {title}<br>"
    popup_content += f"<b>Description:</b> {description[:200]}...<br>"
    
    # Add marker to the cluster
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(marker_cluster)

# Display the map
m

NameError: name 'MarkerCluster' is not defined